In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Multiply, Reshape, Flatten, Concatenate
from keras.optimizers import Adam
from keras.regularizers import l2
import pandas as pd

import json, sys, random, os, datetime, math
print(tf.__version__)
print(keras.__version__)



2.3.0
2.4.0


#Data Pre-processing


In [3]:
from_json_df = pd.read_json('/content/gdrive/My Drive/cmpe256-homework2/video_bundle_data2.json')

In [4]:
from_json_df.head(5)

,bundle_final_price,bundle_url,bundle_price,bundle_name,bundle_id,items,bundle_discount
0,$66.46,http://store.steampowered.com/bundle/450/?utm_...,$73.86,Dharker Studio 2015 Complete,450,"[{'genre': 'Adventure, Indie, RPG', 'item_id':...",10%
1,$20.15,http://store.steampowered.com/bundle/1473/?utm...,$25.87,"Naruto Shippuden Uncut Season 4, Vol. 3",1473,"[{'genre': '', 'item_id': '528360', 'discounte...",22%
2,$20.15,http://store.steampowered.com/bundle/1474/?utm...,$25.87,"Naruto Shippuden Uncut Season 4, Vol. 4",1474,"[{'genre': '', 'item_id': '528380', 'discounte...",22%
3,$15.17,http://store.steampowered.com/bundle/1437/?utm...,$25.35,Pixel Puzzles Ultimate: Starter Kit,1437,"[{'genre': 'Casual, Free to Play, Indie, Simul...",40%
4,$20.15,http://store.steampowered.com/bundle/1466/?utm...,$25.87,"Naruto Shippuden Uncut Season 4, Vol. 1",1466,"[{'genre': '', 'item_id': '527830', 'discounte...",22%


In [5]:
data = from_json_df[['bundle_name','bundle_id','items']]

In [6]:
data.head()

,bundle_name,bundle_id,items
0,Dharker Studio 2015 Complete,450,"[{'genre': 'Adventure, Indie, RPG', 'item_id':..."
1,"Naruto Shippuden Uncut Season 4, Vol. 3",1473,"[{'genre': '', 'item_id': '528360', 'discounte..."
2,"Naruto Shippuden Uncut Season 4, Vol. 4",1474,"[{'genre': '', 'item_id': '528380', 'discounte..."
3,Pixel Puzzles Ultimate: Starter Kit,1437,"[{'genre': 'Casual, Free to Play, Indie, Simul..."
4,"Naruto Shippuden Uncut Season 4, Vol. 1",1466,"[{'genre': '', 'item_id': '527830', 'discounte..."


In [7]:
items_split_array = []
for index, row in data.iterrows():
  for item in row['items']:
    items_split_array.append([item['item_id'], item['item_name'], item['genre'], row['bundle_id']])
df = pd.DataFrame(items_split_array, columns=['item_id', 'item_name', 'genre', 'bundle_id'])
print(df.shape)
df.head()

(3525, 4)


,item_id,item_name,genre,bundle_id
0,326950,Sword of Asumi,"Adventure, Indie, RPG",450
1,331490,Sword of Asumi - Soundtrack,"Adventure, Indie, RPG",450
2,331491,Sword of Asumi - Graphic Novel,"Adventure, Indie, RPG",450
3,331492,Sword of Asumi - Character Creator,"Adventure, Indie, RPG",450
4,348540,Divine Slice of Life,"Adventure, Casual, Indie",450


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3525 entries, 0 to 3524
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_id    3525 non-null   object
 1   item_name  3525 non-null   object
 2   genre      3525 non-null   object
 3   bundle_id  3525 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 110.3+ KB


In [9]:
df.describe()

,bundle_id
count,3525.000000
mean,715.415603
std,396.726058
min,15.000000
25%,362.000000
50%,634.000000
75%,1028.000000
max,1482.000000


In [ ]:
df[df.genre == ''].shape[0]

345

In [ ]:
df['genre'].isin(['Action']).sum(axis=0)

261

In [10]:
df.genre.value_counts()

                                                         345
Action                                                   261
Action, Adventure                                        192
Action, Indie                                            134
Strategy                                                 121
                                                        ... 
Action, Indie, Massively Multiplayer                       1
Action, Adventure, Casual, Racing, Simulation, Sports      1
Casual, Racing                                             1
Action, Casual, Indie, Simulation, Early Access            1
Action, Casual, Indie, RPG                                 1
Name: genre, Length: 197, dtype: int64

In [11]:
df["genre"].replace([""], "Others", inplace=True)

In [12]:
df.genre.value_counts()

Others                                                          345
Action                                                          261
Action, Adventure                                               192
Action, Indie                                                   134
Strategy                                                        121
                                                               ... 
Action, Casual, Indie, Simulation, Early Access                   1
Adventure, Casual, Indie, Simulation, Strategy, Early Access      1
Strategy, Indie, Action                                           1
Casual, Indie, Early Access                                       1
Adventure, Indie, RPG, Simulation, Strategy, Early Access         1
Name: genre, Length: 197, dtype: int64

In [13]:
item_ids = df["item_id"].unique().tolist()
len(item_ids)
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df['itemId'] = df["item_id"].map(item2item_encoded)

df.head()

,item_id,item_name,genre,bundle_id,itemId
0,326950,Sword of Asumi,"Adventure, Indie, RPG",450,0
1,331490,Sword of Asumi - Soundtrack,"Adventure, Indie, RPG",450,1
2,331491,Sword of Asumi - Graphic Novel,"Adventure, Indie, RPG",450,2
3,331492,Sword of Asumi - Character Creator,"Adventure, Indie, RPG",450,3
4,348540,Divine Slice of Life,"Adventure, Casual, Indie",450,4


In [14]:
bundle_ids = df["bundle_id"].unique().tolist()
bundle2bundle_encoded = {x: i for i, x in enumerate(bundle_ids)}
bundle_encoded2bundle = {i: x for i, x in enumerate(bundle_ids)}
df["bundleId"] = df["bundle_id"].map(bundle2bundle_encoded)
df.head()

,item_id,item_name,genre,bundle_id,itemId,bundleId
0,326950,Sword of Asumi,"Adventure, Indie, RPG",450,0,0
1,331490,Sword of Asumi - Soundtrack,"Adventure, Indie, RPG",450,1,0
2,331491,Sword of Asumi - Graphic Novel,"Adventure, Indie, RPG",450,2,0
3,331492,Sword of Asumi - Character Creator,"Adventure, Indie, RPG",450,3,0
4,348540,Divine Slice of Life,"Adventure, Casual, Indie",450,4,0


In [15]:
df.bundleId.min()

0

In [16]:
df.bundleId.max()

614

In [17]:
# split training and validation data
split = np.random.rand(len(df)) < 0.8
train = df[split]
validate = df[~split]
train.head()

,item_id,item_name,genre,bundle_id,itemId,bundleId
0,326950,Sword of Asumi,"Adventure, Indie, RPG",450,0,0
2,331491,Sword of Asumi - Graphic Novel,"Adventure, Indie, RPG",450,2,0
5,352010,Beach Bounce,"Adventure, Casual, Indie",450,5,0
6,390470,Divine Slice of Life - Soundtrack,"Adventure, Casual, Indie",450,6,0
7,394050,Beach Bounce - Soundtrack,"Adventure, Casual, Indie",450,7,0


In [18]:
validate.head()

,item_id,item_name,genre,bundle_id,itemId,bundleId
1,331490,Sword of Asumi - Soundtrack,"Adventure, Indie, RPG",450,1,0
3,331492,Sword of Asumi - Character Creator,"Adventure, Indie, RPG",450,3,0
4,348540,Divine Slice of Life,"Adventure, Casual, Indie",450,4,0
19,528365,Naruto Shippuden Uncut: The Eve of the Five Ka...,Others,1473,19,1
26,528372,Naruto Shippuden Uncut: Declaration of War,Others,1473,26,1


In [19]:
#creating dot matrix which is a sparse matrix which specifies if a bundle has an item
# if a BundleId i has item j, mat[i,j]=1
mat = sp.dok_matrix((train.shape[0], len(df['itemId'].unique())), dtype=np.float32)
for bundle_id, item_id in zip(train['bundleId'], train['itemId']):
    mat[bundle_id, item_id] = 1.0

# Defining Model 


In [20]:
def get_NCF_model(num_bunldes, num_items, latent_dim=8, dense_layers=[64, 32, 16, 8],
              reg_layers=[0, 0, 0, 0], reg_mf=0):

    # input layer
    input_bundle = Input(shape=(1,), dtype='int32', name='bundle_input')
    input_item = Input(shape=(1,), dtype='int32', name='item_input')

    print(reg_mf)
    
    # embedding layer
    mf_bundle_embedding = Embedding(input_dim=num_bundles, output_dim=latent_dim,
                        name='mf_bundle_embedding',
                        embeddings_initializer='RandomNormal',
                        embeddings_regularizer=l2(reg_mf), input_length=1)
    mf_item_embedding = Embedding(input_dim=num_items, output_dim=latent_dim,
                        name='mf_item_embedding',
                        embeddings_initializer='RandomNormal',
                        embeddings_regularizer=l2(reg_mf), input_length=1)
    mlp_bundle_embedding = Embedding(input_dim=num_bundles, output_dim=int(dense_layers[0]/2),
                         name='mlp_bundle_embedding',
                         embeddings_initializer='RandomNormal',
                         embeddings_regularizer=l2(reg_layers[0]), 
                         input_length=1)
    mlp_item_embedding = Embedding(input_dim=num_items, output_dim=int(dense_layers[0]/2),
                         name='mlp_item_embedding',
                         embeddings_initializer='RandomNormal',
                         embeddings_regularizer=l2(reg_layers[0]), 
                         input_length=1)

    # MF latent vector
    mf_bundle_latent = Flatten()(mf_bundle_embedding(input_bundle))
    mf_item_latent = Flatten()(mf_item_embedding(input_item))
    mf_cat_latent = Multiply()([mf_bundle_latent, mf_item_latent])

    # MLP latent vector
    mlp_bundle_latent = Flatten()(mlp_bundle_embedding(input_bundle))
    mlp_item_latent = Flatten()(mlp_item_embedding(input_item))
    mlp_cat_latent = Concatenate()([mlp_bundle_latent, mlp_item_latent])
    
    mlp_vector = mlp_cat_latent
    
    # build dense layer for model
    for i in range(1,len(dense_layers)):
        layer = Dense(dense_layers[i],
                      activity_regularizer=l2(reg_layers[i]),
                      activation='relu',
                      name='layer%d' % i)
        mlp_vector = layer(mlp_vector)

    predict_layer = Concatenate()([mf_cat_latent, mlp_vector])
    result = Dense(1, activation='sigmoid', 
                   kernel_initializer='lecun_uniform',name='result')

    model = Model(inputs=[input_bundle,input_item], outputs=result(predict_layer))

    return model

# get the training samples
def get_train_samples(train_mat, num_negatives):
    bundle_input, item_input, labels = [], [], []
    num_bundle, num_item = train_mat.shape
    for (u, i) in train_mat.keys():
        bundle_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_item)
            while (u, j) in train_mat.keys():
                j = np.random.randint(num_item)
            bundle_input.append(u)
            item_input.append(j)
            labels.append(0)
    return bundle_input, item_input, labels

# hyperparameters

verbose = 1
epochs = 15   
batch_size = 256
latent_dim = 8
dense_layers = [64, 32, 16, 8]
reg_layers = [0, 0, 0, 0]
reg_mf = [0]
num_negatives = 4
learning_rate = 0.001
learner = 'adam'
dataset = 'steam_video'

# loading data
train_mat = mat
    
num_bundles, num_items = train_mat.shape
print('Done loading data!')

Done loading data!


In [21]:
model = get_NCF_model(num_bundles, num_items, latent_dim, dense_layers, reg_layers)
model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
    


0
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bundle_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
mlp_bundle_embedding (Embedding (None, 1, 32)        89408       bundle_input[0][0]               
__________________________________________________________________________________________________
mlp_item_embedding (Embedding)  (None, 1, 32)        89536       item_input[0][0]                 
_____________________________________________________________________________________

#Training

In [22]:

bundle_input, item_input, labels = get_train_samples(train_mat, num_negatives)

# training
hist = model.fit([np.array(bundle_input), np.array(item_input)], np.array(labels), 
                 batch_size=batch_size, epochs=epochs, verbose=verbose, shuffle=True)

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


55/55 [==============================] - 0s 4ms/step - loss: 0.6282 - accuracy: 0.7967
Epoch 2/15
55/55 [==============================] - 0s 4ms/step - loss: 0.5104 - accuracy: 0.8000
Epoch 3/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4889 - accuracy: 0.8000
Epoch 4/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4743 - accuracy: 0.8000
Epoch 5/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4622 - accuracy: 0.8000
Epoch 6/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4513 - accuracy: 0.8000
Epoch 7/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4402 - accuracy: 0.8000
Epoch 8/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4261 - accuracy: 0.8013
Epoch 9/15
55/55 [==============================] - 0s 4ms/step - loss: 0.4032 - accuracy: 0.8181
Epoch 10/15
55/55 [==============================] - 0s 4ms/step - loss: 0.3626 - accuracy: 0.8350
Epoch 11/15
55/55 [===========

In [23]:
model_file = '%s_NCF_%d_%s.h5' % (dataset, latent_dim, str(dense_layers))
model.save(model_file, overwrite=True)

#Evaluation


In [44]:
from keras.models import load_model
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

# this is a sample bundleid to predict
desired_bundle_id = 300
model_path = 'steam_video_NCF_8_[64, 32, 16, 8].h5'
print('using model: %s' % model_path)
model = load_model(model_path)
print('Loaded model!')

mlp_bundle_embedding_weights = (next(iter(filter(lambda x: x.name == 'mlp_bundle_embedding', model.layers))).get_weights())

# get the latent embedding for your desired bundle
bundle_latent_matrix = mlp_bundle_embedding_weights[0]
one_bundle_vector = bundle_latent_matrix[desired_bundle_id,:]
one_bundle_vector = np.reshape(one_bundle_vector, (1,32))

print('\nPerforming kmeans to find the nearest bundles...')
# get 50 similar bundles
kmeans = MiniBatchKMeans(n_clusters=50, random_state=0, verbose=1).fit(bundle_latent_matrix)
desired_bundle_label = kmeans.predict(one_bundle_vector)
bundle_label = kmeans.labels_
neighbors = []
for bundle_id, bundle_label in enumerate(bundle_label):
    if bundle_label == desired_bundle_label:
        neighbors.append(bundle_id)
print('Found {0} neighbor bundles/games.'.format(len(neighbors))) 

using model: steam_video_NCF_8_[64, 32, 16, 8].h5
Loaded model!

Performing kmeans to find the nearest bundles...
Init 1/3 with method: k-means++
Inertia for init 1/3: 20.822716
Init 2/3 with method: k-means++
Inertia for init 2/3: 20.318680
Init 3/3 with method: k-means++
Inertia for init 3/3: 20.248512
Minibatch iteration 1/2800: mean batch inertia: 0.089012, ewa inertia: 0.089012 
Minibatch iteration 2/2800: mean batch inertia: 0.082096, ewa inertia: 0.088517 
Minibatch iteration 3/2800: mean batch inertia: 0.086896, ewa inertia: 0.088401 
Minibatch iteration 4/2800: mean batch inertia: 0.084992, ewa inertia: 0.088157 
Minibatch iteration 5/2800: mean batch inertia: 0.080194, ewa inertia: 0.087587 
Minibatch iteration 6/2800: mean batch inertia: 0.082738, ewa inertia: 0.087240 
Minibatch iteration 7/2800: mean batch inertia: 0.089807, ewa inertia: 0.087424 
Minibatch iteration 8/2800: mean batch inertia: 0.077271, ewa inertia: 0.086697 
Minibatch iteration 9/2800: mean batch inertia

In [45]:
# get the games in similar bundles' items
games = []
for bundle_id in neighbors:
    games += list(df[df['bundleId'] == int(bundle_id)]['itemId'])
print('Found {0} neighbor items from these games.'.format(len(games))) 

games_arr = np.full(len(games), desired_bundle_id, dtype='int32')
bundles = np.array(games, dtype='int32')

print('\nRanking most likely games using the NeuMF model...')
# and predict games for bundle
results = model.predict([games_arr,bundles],batch_size=100, verbose=0) 
results = results.tolist()
print('Ranked the games!')

Found 280 neighbor items from these games.

Ranking most likely games using the NeuMF model...
Ranked the games!


In [46]:
results_df = pd.DataFrame(np.nan, index=range(len(results)), columns=['probability','item_name', 'genre'])
print(results_df.shape)

# loop through and get the probability (of being in the bundle according to my model), the game, and the genre  
for i, prob in enumerate(results):
    # print('i: {0} prob: {1}'.format(i,prob))
    results_df.loc[i] = [prob[0], df[df['itemId'] == i].iloc[0]['item_name'], df[df['itemId'] == i].iloc[0]['genre']]
results_df = results_df.sort_values(by=['probability'], ascending=False)

results_df.head(3)

(280, 3)


,probability,item_name,genre
100,0.994631,Black Sand Drift Collector's Edition Content,"Casual, Indie"
172,0.994204,Naruto Shippuden Uncut: S10E09,Others
156,0.994204,Overlord: Fellowship of Evil,"Action, Adventure"


In [47]:
df[df['bundleId'] == 300].head(5)

,item_id,item_name,genre,bundle_id,itemId,bundleId
1474,438010,PulseCharge,"Action, Adventure, Indie",480,1301,300
1475,438030,Dead6hot,"Action, Casual",480,1302,300
1476,444480,Broken Dreams,"Casual, Indie",480,1303,300
1477,463220,Dungeon of Zolthan,"Action, Adventure, Indie",480,1304,300
1478,491650,Reset 1-1,Indie,480,1305,300


In [ ]:
#save df to json for inference
df.to_json(r'/content/origin_steam_video_df.json')

In [ ]:
load_df = pd.read_json('/content/origin_steam_video_df.json')
load_df.head()

In [48]:
from google.colab import files
files.download('/content/steam_video_NCF_8_[64, 32, 16, 8].h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/origin_steam_video_df.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>